In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import os
import cv2
import numpy as np

In [12]:
parasitized_input_dir = os.listdir('cell_images/Parasitized') 
uninfected_input_dir = os.listdir('cell_images/Uninfected')
img_height = 50
img_width = 50
batch_size = 32
no_of_epochs = 5

In [13]:
parasitized_input_dir

['C97P58ThinF_IMG_20150917_145555_cell_182.png',
 'C182P143NThinF_IMG_20151201_172216_cell_139.png',
 'C176P137NThinF_IMG_20151201_121808_cell_132.png',
 'C78P39ThinF_IMG_20150606_103459_cell_172.png',
 'C129P90ThinF_IMG_20151004_134700_cell_3.png',
 'C142P103ThinF_IMG_20151005_221931_cell_191.png',
 'C80P41ThinF_IMG_20150817_111544_cell_129.png',
 'C137P98ThinF_IMG_20151005_161449_cell_2.png',
 'C56P17thinF_IMG_20150728_160623_cell_146.png',
 'C50P11thinF_IMG_20150724_120716_cell_154.png',
 'C167P128ReThinF_IMG_20151201_105354_cell_229.png',
 'C133P94ThinF_IMG_20151004_155518_cell_102.png',
 'C59P20thinF_IMG_20150803_113430_cell_25.png',
 'C131P92ThinF_IMG_20151004_151001_cell_123.png',
 'C99P60ThinF_IMG_20150918_141314_cell_138.png',
 'C135P96ThinF_IMG_20151005_124031_cell_161.png',
 'C99P60ThinF_IMG_20150918_141129_cell_148.png',
 'C39P4thinF_original_IMG_20150622_114804_cell_34.png',
 'C48P9thinF_IMG_20150721_160406_cell_233.png',
 'C45P6ThinF_IMG_20151130_160241_cell_207.png',
 'C

In [14]:
image_data = [] 
labels = []
# for reading images of parasitized cells
for p in parasitized_input_dir:
    try:
        img = cv2.imread('cell_images/Parasitized/' + p)
        img = cv2.resize(img, (img_width, img_height)) # all images will be resized to 50*50
        image_data.append(img)
        labels.append(1)
    except:
        print("Error!!")

Error!!


In [15]:
for u in uninfected_input_dir:
    try:
        img = cv2.imread('cell_images/Uninfected/' + u)
        img = cv2.resize(img, (img_width, img_height))
        image_data.append(img)
        labels.append(0)
    except:
        print("Error!!")

Error!!


In [16]:
image_data = np.array(image_data)
labels = np.array(labels)

In [18]:
print(labels.shape)

(27558,)


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(image_data, labels, test_size=0.2, random_state=101)

In [20]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=15, shear_range=0.2, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
train_datagen.fit(X_train)
test_datagen.fit(X_test)

In [25]:
#let's create an architecture for CNN
model=Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(img_height, img_width, 3)))
model.add(Conv2D(16, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(16, (3, 3), input_shape=(img_height, img_width, 3)))
model.add(Conv2D(16, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Conv2D(16, (3, 3), input_shape=(img_height, img_width, 3)))
model.add(Conv2D(16, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
          

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 16)        2320      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 16)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 16)          2320      
__________

In [33]:
model.fit_generator(train_datagen.flow(X_train, Y_train), 
                    steps_per_epoch=len(X_train)//batch_size,
                    epochs=no_of_epochs,
                    validation_data=test_datagen.flow(X_test, Y_test),
                    validation_steps=len(X_test)//batch_size
                   )

Epoch 1/5
688/688 [==============================] - 40s 58ms/step - loss: 0.1931 - acc: 0.9371 - val_loss: 0.1702 - val_acc: 0.9395
Epoch 2/5
688/688 [==============================] - 40s 57ms/step - loss: 0.1870 - acc: 0.9388 - val_loss: 0.1724 - val_acc: 0.9391
Epoch 3/5
688/688 [==============================] - 39s 57ms/step - loss: 0.1863 - acc: 0.9387 - val_loss: 0.1772 - val_acc: 0.9418
Epoch 4/5
688/688 [==============================] - 40s 58ms/step - loss: 0.1833 - acc: 0.9384 - val_loss: 0.1740 - val_acc: 0.9391
Epoch 5/5
688/688 [==============================] - 40s 58ms/step - loss: 0.1809 - acc: 0.9395 - val_loss: 0.1645 - val_acc: 0.9440
